In [45]:
import pandas as pd
from konlpy.tag import Okt
import re
from multiprocessing import Pool
from tqdm import tqdm
tqdm.pandas()

In [46]:
df = pd.read_csv('../crawling_data/reviews_2016.csv')
df.head(20)

,reviews,titles,years
0,첫 장편 영화로 수많은 영화제에서 관심과 주목을 받은 나지 아부 노워감독의 영화가 ...,디브 (Theeb),2016
1,지금은 석유자원으로 중동 사막지역이 황금의 땅이 되었지만 그 덕분에 세계열강의 침...,디브 (Theeb),2016
2,"아랍영화 디브를 보앗네요\r\n원제가 [THEEB, WolF] 즉 늑대 디브라서 ...",디브 (Theeb),2016
3,늑대가 되어야 했던 소년\r\n<디브>\r\n“당신을 놓아주지 않을 것이다”\r\n...,디브 (Theeb),2016
4,\r\n아카데미시상식과 베니스국제영화제가 선택한 한 소년의 강렬한 운명! 붉은 사...,디브 (Theeb),2016
5,시들어가는 중...\r\n시든다는 표현이 이렇게 잘 어울리는 날씨가 있을까요?\r\...,디브 (Theeb),2016
6,디브\r\n감독\r\n나지 아부 노워\r\n출연\r\n자시르 에이드 알휴타트\r\n...,디브 (Theeb),2016
7,"\r\n\r\n어떤 기술이 빚어낸 방아쇠는 탄환, 총알을 밀어내고\r\n총알은 그...",디브 (Theeb),2016
8,\r\n \r\n이웃 블로거 한마루 님의 이벤트에 당첨되어 보고 온 영화 디브.\...,디브 (Theeb),2016
9,2016년 Movie Review NO.260\r\n \r\n\r\n 디브\r\nT...,디브 (Theeb),2016


In [47]:
df.shape

(2347, 3)

In [48]:
# # 정규식 사용해보기
# a = '1a25ds94k|가나다라마'
# s = re.sub('[^가-힣 ]', '', a)
# print(s)

In [49]:
# # 첫번째 문장에 사용해보기
# print(df.iloc[0, 0])
# print('='*100)
# sentence = re.sub('[^가-힣 ]', '', df.iloc[0, 0])
# print(sentence)

In [50]:
okt = Okt()
# token = okt.pos(sentence)
# print(token)

In [51]:
# # 데이터 프레임
# df_token = pd.DataFrame(token, columns=['word', 'class']) 
# print(df_token.head(20)) 

In [52]:
# # 조건 인덱싱
# df_cleaned_token = df_token[(df_token['class'] == 'Noun') | 
#                             (df_token['class'] == 'Verb') |
#                             (df_token['class'] == 'Adjective')]
# print(df_cleaned_token.head(20))

In [53]:
# a = [1, 2, 3]
# b = [4, 5, 6]
# c = a + b
# print(c)

In [54]:
stopwords = pd.read_csv('../datasets/stopwords.csv', index_col=0)
print(stopwords)

    stopword
0          아
1          휴
2        아이구
3        아이쿠
4        아이고
..       ...
783      없다는
784      않을까
785       있지
786      있으며
787      그대로

[787 rows x 1 columns]


In [55]:
# 또 다른 불용어 추가
movie_stopwords = ['영화', '배우', '감독']
stopwords_list = list(stopwords.stopword) + movie_stopwords

In [56]:
# # 불용어 처리
# words = []
# for word in df_cleaned_token['word']:
#     if len(word) > 1:
#         if word not in stopwords_list:
#             words.append(word)
# print(words)

In [57]:
# # 한 문장으로
# cleaned_sentence = ' '.join(words)
# print(cleaned_sentence)

In [58]:
#count = 0
cleaned_sentences=[]
for sentence in tqdm(df.reviews):
    sentence = re.sub('[^가-힣 ]', '', sentence)
    token = okt.pos(sentence, stem=True)  # 단어의 원형 반환   
    df_token = pd.DataFrame(token, columns=['word', 'class']) 
    df_cleaned_token = df_token[(df_token['class'] == 'Noun') | (df_token['class'] == 'Verb') | (df_token['class'] == 'Adjective')]
    words = [word for word in df_cleaned_token['word'] if word not in stopwords_list and len(word) > 1]
    cleaned_sentence = ' '.join(words)
    cleaned_sentences.append(cleaned_sentence)

df['cleaned_sentences'] = cleaned_sentences
df



100%|██████████| 2347/2347 [02:10<00:00, 18.04it/s]


,reviews,titles,years,cleaned_sentences
0,첫 장편 영화로 수많은 영화제에서 관심과 주목을 받은 나지 아부 노워감독의 영화가 ...,디브 (Theeb),2016,장편 영화로 수많다 영화제 관심 주목 받다 나다 아부 놓다 국내 개봉 모로 흥미롭다...
1,지금은 석유자원으로 중동 사막지역이 황금의 땅이 되었지만 그 덕분에 세계열강의 침...,디브 (Theeb),2016,유자 중동 사막 지역 황금 되어다 덕분 세계 강의 침략 구실 되어다 세계대전 당시 ...
2,"아랍영화 디브를 보앗네요\r\n원제가 [THEEB, WolF] 즉 늑대 디브라서 ...",디브 (Theeb),2016,아랍 디브 원제 늑대 브라 늘다 대가 소년 이여기 늑대 이기 늑대 수박 업다 환경 ...
3,늑대가 되어야 했던 소년\r\n<디브>\r\n“당신을 놓아주지 않을 것이다”\r\n...,디브 (Theeb),2016,늑대 되어다 소년 디브 놓다 늑대 되어다 소년 강렬하다 눈빛 국내 개봉 확정 시선 ...
4,\r\n아카데미시상식과 베니스국제영화제가 선택한 한 소년의 강렬한 운명! 붉은 사...,디브 (Theeb),2016,아카데미 시상식 베니스 국제 영화제 선택 소년 강렬하다 운명 붉다 사막 집다 삼키다...
...,...,...,...,...
2342,트뤼포가 히치콕을 인터뷰한 녹음과 히치콕의 주요 영화들의 장면을 보여주고 현시대의 ...,히치콕 트뤼포 (Hitchcock/Truffaut),2016,트뤼포 히치콕 인터뷰 녹음 히치콕 주요 장면 보여주다 현시 대의 비평가 히치콕 어떻...
2343,히치콕 트뤼포 (2015)\r\nHitchcock/Truffaut\r\n다큐멘터리 ...,히치콕 트뤼포 (Hitchcock/Truffaut),2016,히치콕 트뤼포 다큐멘터리 개봉 미국 프랑스 켄트 존스 주연 알프레드 히치콕 본인 자...
2344,"히치콕 트뤼포\r\n감독\r\n켄트 존스\r\n출연\r\n알프레드 히치콕, 프랑수아...",히치콕 트뤼포 (Hitchcock/Truffaut),2016,히치콕 트뤼포 켄트 존스 출연 알프레드 히치콕 프랑수아 트뤼포 데이빗 핀처 마틴 스...
2345,"감독 : 켄트 존스\r\n주연 : 프랑수아 트뤼포 , 마티유 아말릭\r\n 영화...",히치콕 트뤼포 (Hitchcock/Truffaut),2016,켄트 존스 주연 프랑수아 트뤼포 마티유 아말릭 영화계 거장 남기다 위대하다 세기 거...


In [60]:
df = df[['titles', 'cleaned_sentences']]
print(df.info())
df.to_csv('../crawling_data/cleaned_review_2016.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2347 entries, 0 to 2346
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   titles             2347 non-null   object
 1   cleaned_sentences  2347 non-null   object
dtypes: object(2)
memory usage: 36.8+ KB
None


In [ ]:
    # # 진행 상황 확인
    # count +- 1
    # if count % 10 == 0:
    #     print('.', end='')
    # if count % 100 == 0:
    #     print('')
    
    # words = []
    # for word in df_cleaned_token['word']:
    #     if len(word) > 1:
    #         if word not in stopwords_list:
    #             words.append(word)

In [ ]:
# def review_to_words(raw_data):
#     cleaned_sentences=[]
#     letters_only = re.sub('[^가-힣 ]', '', raw_data)
#     token = okt.pos(letters_only, stem=True) 
#     token = pd.DataFrame(token, columns=['word', 'class'])
#     cleaned_token = token[(token['class'] == 'Noun') | 
#                           (token['class'] == 'Verb') |
#                           (token['class'] == 'Adjective')]  
#     stops = set(stopwords_list)
#     meaningful_words = [w for w in df_cleaned_token['word'] if not w in stops and len(w) > 1] # 불용어 처리
#     cleaned_sentence = ' '.join(meaningful_words) # 한 문장으로
#     cleaned_sentences.append(cleaned_sentence)
    
#     return(cleaned_sentences)
   
# df['cleaned_reviews'] = review_to_words(df.reviews)        
# df

In [ ]:
# def _apply_df(args):
#     df, func, kwargs = args
#     return df.apply(func, **kwargs)

# def apply_by_multiprocessing(df, func, **kwargs):
#     # 키워드 항목 중 workers 파라메터를 꺼냄
#     workers = kwargs.pop('workers')
#     # 위에서 가져온 workers 수로 프로세스 풀을 정의
#     pool = Pool(processes=workers)
#     # 실행할 함수와 데이터프레임을 워커의 수 만큼 나눠 작업
#     result = pool.map(_apply_df, [(d, func, kwargs)
#             for d in np.array_split(df, workers)])
#     pool.close()
#     # 작업 결과를 합쳐서 반환
#     return pd.concat(list(result))

# cleaned_sentences = apply_by_multiprocessing(\
#     train['review'], review_to_words, workers=8)  
